In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order

plt.rc("figure", figsize=(16,8))
plt.rc("font", size=14)
from datetime import datetime

In [3]:
#import vaccine distro
df = pd.read_csv (r'https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/raw_data/vaccine_data_us_state_timeline.csv')

#import state populations
state_pops = pd.read_csv(r'state_thresholds.csv')


In [4]:
#clean and transpose vaccine distro 

df['date'] = pd.to_datetime(df['date'])
doses_admin = df.pivot(index='Province_State', columns='date',values='doses_admin_total')
d_a = doses_admin.transpose()
d_a.head()

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,Bureau of Prisons,California,Colorado,Connecticut,Delaware,...,Texas,Utah,Vermont,Veterans Health Administration,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
date,,,,,,,,,,,,,,,,,,,,,
2020-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#get list of states to loop through
states = state_pops['State']

#set up empty dataframes for the loop
immunity = pd.DataFrame(columns=['vaccinated', 'state'], index=pd.to_datetime([]))
vacc = pd.DataFrame(columns=['vaccinated', 'state'], index=pd.to_datetime([]))

In [9]:
#get date for herd immunity from states

immunity = pd.DataFrame(columns=['vaccinated', 'state'], index=pd.to_datetime([]))

for state in states:
    d_state = d_a[state]
    dates = pd.date_range('2021-01-20', periods=len(d_state), freq="D")
    endog = pd.Series(d_state)
    endog.index = pd.DatetimeIndex(endog.index).to_period('D')
    endog = endog.dropna()
    selection_res = ar_select_order(endog, maxlag=1, old_names=True, glob=True, seasonal=False)
    pandas_ar_res = selection_res.model.fit()
    pred = pandas_ar_res.predict(start='2021-01-20', end='2045-12-31').round(0)
    threshold = state_pops.loc[state_pops['State'] == state, 'Threshold'].iloc[0]
    for num in pred:
        if(num > threshold):
            vacc = pd.DataFrame(columns=['vaccinated', 'state'], index=pd.to_datetime([]))
            vaccination_day = pred.loc[pred==num]
            vacc['vaccinated'] = vaccination_day
            vacc['state'] = state
            immunity = immunity.append(vacc)
            break

In [10]:
immunity

,vaccinated,state
2021-05-29,7374301.0,Alabama
2021-05-20,1108648.0,Alaska
2021-04-27,10934061.0,Arizona
2021-05-01,60228699.0,California
2021-04-26,8659207.0,Colorado
2021-05-17,5439954.0,Connecticut
2021-05-15,1493340.0,Delaware
2021-04-28,33001127.0,Florida
2021-05-31,15965827.0,Georgia
2021-05-26,2139788.0,Hawaii


In [11]:
immunity.to_csv(r'predicted_immunity.csv', index=True, header=None)